# Clean Data

This notebook processes and cleans parsed news article data.  

The main steps are:

1. **Standardizing values** (text cleaning, formatting, and handling missing data)
2. **Filtering** (removing short articles and unwanted titles)
3. **Deduplication** (exact matches and near-duplicates)
4. **Final clean-up** (column selection, reordering, and saving)

In [1]:
import os
import pandas as pd
import re
from ftfy import fix_text
import pickle

from utils import standardize_text, add_duplicate_flags

# Input and output paths
input_path = "../data/processed/parsed_articles.pkl"
output_path = "../data/processed/cleaned_articles.pkl"

# Load data
df_articles = pd.read_pickle(input_path)

# number of articles expected
expected_n_articles = 500 * 86 + 305 + 361

# Tracking summary counts
summary = []
def log_count(step_name, before, after):
    summary.append({
        "Step": step_name,
        "Before": before,
        "After": after,
        "Dropped": before - after
    })
    
# Record initial load
summary.append({
    "Step": "Initial load",
    "Before": expected_n_articles,
    "After": df_articles.shape[0],
    "Dropped": df_articles.shape[0] - expected_n_articles
})

## 1. Standardize Values

Here we:

* Fill missing values in key columns
* Convert article length to integers
* Clean text fields (normalize whitespace, fix broken characters, fix text with ftfy)
* Standardize titles (remove punctuation and stop words)
* Remove trailing "Load-Date" metadata from article bodies
* Group publishers into consistent categories

In [2]:
# Load title stop words
with open("title_stop_words.pkl", "rb") as f:
    title_stop_words = pickle.load(f)

In [3]:
def clean_text(text):
    text = text.replace(u'\xa0', u' ')
    return fix_text(text).strip(" \n")

def remove_load_date(text: str) -> str:
    """
    Remove 'Load-Date: <Month day, year>' if present at the end of the text.
    """
    # Regex: match optional whitespace, then 'Load-Date:', then date, till the very end
    pattern = r'\s*Load-Date:\s+[A-Za-z]+\s+\d{1,2},\s+\d{4}\s*$'
    return re.sub(pattern, '', text)
    
def convert_length(length): # see Guo code
    length = int(length.strip(" ")[:-6])
    return length

def obtain_publisher_group(source_file):
    folder = os.path.dirname(source_file)
    if folder == "NYT":
        return "New York Times"
    elif folder == "Other publishers":
        return "Other publisher"
    else:
        raise ValueError(f"Unknown publisher group for folder: {folder}")

In [4]:
# Fill missing values
df_articles["section"] = df_articles["section"].fillna("") 
df_articles["length"] = df_articles["length"].fillna("0 words")

# Convert length string to integer
df_articles["length"] = df_articles["length"].apply(convert_length)

In [5]:
# Clean text columns
txt_cols = ['title', 'publisher', 'section', 'body']
for txt_col in txt_cols:
    df_articles[txt_col] = df_articles[txt_col].apply(clean_text)

In [6]:
# Standardize titles
df_articles['title_stand'] = df_articles['title'].apply(lambda x: standardize_text(x, title_stop_words))

# Remove trailing load date
df_articles['body'] = df_articles['body'].apply(remove_load_date)

In [7]:
# Assign publisher group
df_articles['publisher_group'] = df_articles['source_file'].apply(obtain_publisher_group)

## 2. Filters

We filter articles to:
* Remove very short articles (fewer than 100 words)
* Exclude recurring recommendation list titles
* Remove articles whose titles start with specific patterns

In [8]:
# Initial count
before = df_articles.shape[0]
print(f"Starting number of articles: {before}.")

Starting number of articles: 43666.


In [9]:
# Filter 1: Minimum length
df_articles = df_articles.loc[df_articles['length'] >= 100]
after = df_articles.shape[0]
log_count("Filter: Min length ≥ 100 words", before, after)
print(f"Dropped {before - after} articles. Net number of articles: {df_articles.shape[0]}")

Dropped 109 articles. Net number of articles: 43557


In [10]:
# Filter 2: Specific titles
titles_reccomendation_lists = ['new & noteworthy paperbacks',
                             'paperback row',
                             'art',
                             'four stars: superior. three stars: good. two stars: average. one star: poor. d (',
                             'movie guide']

In [11]:
df_articles = df_articles.loc[~df_articles['title'].str.lower().isin(titles_reccomendation_lists)]
before = after
after = df_articles.shape[0]
log_count("Filter: Exact title match list", before, after)
print(f"Dropped {before - after} articles. Net number of articles: {df_articles.shape[0]}")

Dropped 292 articles. Net number of articles: 43265


In [12]:
# Filter 3: Titles starting with certain patterns
startswith_filter = ('best sellers:', 'four stars:')
df_articles = df_articles[
    ~df_articles['title'].str.lower().str.startswith(startswith_filter)
]
before = after
after = df_articles.shape[0]
log_count("Filter: Title startswith patterns", before, after)
print(f"Dropped {before - after} articles. Net number of articles: {df_articles.shape[0]}")

Dropped 241 articles. Net number of articles: 43024


## 3. Deduplication

We remove:
* **Exact duplicates** (keeping earliest date)
* **Near-duplicates** (based on match in standardardized title and body similarity score above 90%)

In [13]:
# Exact duplicates
# Shuffle for randomness
df_articles = df_articles.sample(frac=1, random_state=42).reset_index(drop=True)

# Sort by date and drop duplicates
df_articles = (
    df_articles
    .sort_values(by='date', ascending=True)
    .drop_duplicates(subset=["body", "title"], keep='first')
)

before = after
after = df_articles.shape[0]
log_count("Deduplication: Exact matches", before, after)
print(f"Dropped {before - after} articles. Net number of articles: {df_articles.shape[0]}")

Dropped 551 articles. Net number of articles: 42473


In [14]:
# Near-duplicates
thresholds = [0.90]
df_articles = add_duplicate_flags(df_articles, "title_stand", thresholds)

Obtaining near-duplicates flag for 0.9 threshold


In [15]:
df_articles = df_articles.loc[~df_articles['is_near_duplicate_90']]
before = after
after = df_articles.shape[0]
log_count("Deduplication: Near-duplicates (90%)", before, after)
print(f"Dropped {before - after} articles. Net number of articles: {df_articles.shape[0]}")

Dropped 1179 articles. Net number of articles: 41294


## 4. Final Clean-up & Save

We:
* Keep only selected columns
* Rename publisher group to publisher
* (TODO) Separate letters to the editor
* Sort data and reset index
* Save cleaned DataFrame

In [16]:
# Select and rename columns
cols = ['title', 'publisher_group', 'date', 'section', 'body', 'source_file']
df_articles = df_articles[cols].rename(columns = {"publisher_group": "publisher"})

In [17]:
# TODO: Separate letters to the editor

In [18]:
# Sort and reset index
df_articles = df_articles.sort_values(by='date', ascending=False).reset_index(drop=True)

In [19]:
# Save cleaned data
df_articles.to_pickle(output_path)

In [20]:
# Preview
df_articles.head()

,title,publisher,date,section,body,source_file
0,"Under a Highway in Rio, a Dance Style Charms a...",New York Times,2024-12-31,WORLD; americas,"Trucks, buses and cars rumbled overhead, drown...",NYT/1.DOCX
1,These were the big stories in arts and culture...,Other publisher,2024-12-31,WHAT TO KNOW,The arts in Dayton continued to thrive in 2024...,Other publishers/Files (500) (1).DOCX
2,She Exalted The Beauty Of Dance,New York Times,2024-12-31,Section C; Column 0; The Arts/Cultural Desk; P...,She was The New Yorker's first dance critic. H...,NYT/1.DOCX
3,A Well-Documented Childhood. A Very Private Life.,New York Times,2024-12-31,Section A; Column 0; National Desk; Pg. 16,Jimmy Carter's daughter had an extraordinary a...,NYT/1.DOCX
4,"Congressional pay, minimum wage stagnant for y...",Other publisher,2024-12-31,OPINION; Pg. A13,ABSTRACT\nMembers of Congress have not seen a ...,Other publishers/Files (500) (1).DOCX


# 5. Summary Table of Counts

In [21]:
summary_df = pd.DataFrame(summary)
display(summary_df)

,Step,Before,After,Dropped
0,Initial load,43666,43666,0
1,Filter: Min length ≥ 100 words,43666,43557,109
2,Filter: Exact title match list,43557,43265,292
3,Filter: Title startswith patterns,43265,43024,241
4,Deduplication: Exact matches,43024,42473,551
5,Deduplication: Near-duplicates (90%),42473,41294,1179
